# Exploration de mini Dall-E

## 1. Comment les textes images & textes sont-ils pré-traités / encodés ?

Le pré-traitement des textes et images sont répartis dans de nombreuses classes et fichiers Python. Essentiellement, il consiste à convertir le texte en une suite de tokens (des entiers qui représentent des catégories discrètes) avec `BartTokenizerFast` et à faire de même avec les images avec `VQModel`. Chacun de ces modèles possède en théorie un dictionnaire associé, qui associe à chaque token un vecteur dans $\mathbb{R}^d$, construit de telle façon que la proximité entre deux vecteurs traduise une proximité entre les tokens correspondant. Contrairement à ce qu'une lecture trop rapide des articles de Dall-E et mini Dall-E pourrait laisser penser, ces vecteurs sont complètement ignorés. Les modèles doivent donc ré-apprendre la proxmité entre "father" et "mother" ou entre "rose" et "roses".

Dans le détail, voici les étapes du pré-traitement dans l'ordre de l'investigation:

In [ ]:
# dalle-mini/tools/train.py, lignes 539-544

dataset.preprocess( # <------------------------------------------------ preprocess ?
    tokenizer = tokenizer, # <----------------------------------------- tokenizer ?
    decoder_start_token_id = model.config.decoder_start_token_id,
    normalize_text = model.config.normalize_text,
    max_length = model.config.max_text_length,
)

La recherche de `tokenizer` est la plus courte:

In [ ]:
# dalle-mini/tools/train.py, lignes 488-492

# load tokenizer
tokenizer = DalleBartTokenizer.from_pretrained(
    artifact_dir,
    use_fast=True,
)

# dalle-mini/tools/train.py, ligne 49-54

from dalle_mini.model import DalleBartTokenizer

# dalle-mini/model/__init__.py, ligne 4

from .tokenizer import DalleBartTokenizer

# dalle-mini/model/tokenizer.py

# En subtance:
# DalleBartTokenizer = BartTokenizerFast + surcouche Weights and biases

La rechecherche de la méthode `preprocess()` est plus fastidieuse.

In [ ]:
# dalle-mini/src/dalle-mini/data.py, lignes 88 et suivantes
# largement modifié par soucis de clarté

def preprocess(self, tokenizer, decoder_start_token_id, normalize_text, max_length):

    def partial_preprocess_function = partial( # lignes 125-132
        preprocess_function, # <----------------------------------- preprocess_function ?
        tokenizer=tokenizer,
        text_column=self.text_column, # <-------------------------- defaults to "caption"
        encoding_column=self.encoding_column, # <------------------ defaults to "embedding"
        max_length=max_length,
        decoder_start_token_id=decoder_start_token_id,
    )

    # dalle-mini/src/dalle-mini/data.py, lignes 133-154, édité par soucis de clarté
    ds = ds.map(partial_preprocess_function, batched=True) # <----- ds ?

    # dalle-mini/src/dalle-mini/data.py, lignes 100-122, édité par soucis de clarté
    ds = self.train_dataset # OU ds = self.eval_dataset

# dalle-mini/src/dalle-mini/data.py, lignes 255-286, édité par soucis de clarté

def preprocess_function(
    examples, # <--------------------------------------------------- examples = 1 element from ds
    tokenizer,
    text_column,
    encoding_column,
    max_length,
    decoder_start_token_id,
):
    inputs = examples[text_column] # <------------------------------ this element must have "caption" 

    labels = examples[encoding_column] # <-------------------------- this element must have "embedding" 
    labels = np.asarray(labels)

    model_inputs                      = tokenizer(inputs, ... )
    model_inputs["labels"]            = labels
    model_inputs["decoder_input_ids"] = shift_tokens_right(labels, decoder_start_token_id)
    # shift_tokens_right() prepends the <bos> token and removes the last one

    return model_inputs

Maintenant reste à savoir comment ce `self.train_dataset` est constitué!

In [ ]:
# dalle-mini/tools/dataset/encode_dataset.ipynb, édité par soucis de clarté

# pas de numéros de lignes :(
pd.DataFrame.from_dict(
    {"caption": all_captions, "encoding": all_encoding} # <-------- all_captions ? all_encodings ?
)

# pas de numéros de lignes :(
for (images, captions) in dataloader :
    images = images.numpy()
    encoded = p_encode(images, vqgan_params) # <------------------- p_encode ?
    encoded = encoded.reshape(-1, encoded.shape[-1])
    all_captions.extend(captions)
    all_encoding.extend(encoded.tolist())

# pas de numéros de lignes :(
@partial(jax.pmap, axis_name="batch")
def p_encode(batch, params):
    # Not sure if we should `replicate` params, does not seem to have any effect
    _, indices = vqgan.encode(batch, params=params) # <------------ vqgan ?
    return indices

# pas de numéros de lignes :( 
vqgan = VQModel.from_pretrained("flax-community/vqgan_f16_16384")

## 2. Comment formater les tokens texte + image pour les passer à Bart?



In [ ]:
def train_step(state, batch, ...): # <---------------------------- state object contains info about the
#                                                                  current state of the model

    def compute_loss(params, minibatch, dropout_rng): # <--------- minibatch is derived from batch
        
            minibatch, labels = minibatch.pop("labels")
            logits = state.apply_fn( # <-------------------------- apply_fn ?
                **minibatch, .... # <----------------------------- minibatch ?
            )[0]
            return loss_fn(logits, labels)
    
    grad_fn = jax.value_and_grad(compute_loss)
    # code continues to accumalate gradient accross the batch

    # update state
    loss, grads = loss_grad
    state = state.apply_gradients(
        grads=grads,
        dropout_rng=dropout_rng,
        train_time=state.train_time + delta_time,
        train_samples=state.train_samples + batch_size_per_step,
    )

    metrics = {
        "loss": loss,
        "learning_rate": learning_rate_fn(state.step),
    }

    return state, metrics

What is this `apply_fn` method? And what do `minibatch`'s look like?

In [ ]:
# https://github.com/borisdayma/dalle-mini/blob/main/tools/train/train.py, lignes 708-714
def init_state( ... ):
    return TrainState.create(
        apply_fn = model.__call__,
        ...
    )

# https://github.com/borisdayma/dalle-mini/blob/main/tools/train/train.py, lignes 493-509
model = DalleBart(...)

In [ ]:
# minibatch comes from batch
# I assume it has the same structure

def get_minibatch(batch, grad_idx):
    return jax.tree_map(
        lambda x: jax.lax.dynamic_index_in_dim(x, grad_idx, keepdims=False),
        batch,
    )

# https://github.com/borisdayma/dalle-mini/blob/main/tools/train/train.py, lignes 1057-1063

for batch in train_loader: #  <--------------------------------------------- trainloader ?
    
    # ligne 1085
    state, train_metrics = p_train_step(state, batch, delta_time) # <------- p_train_step ?

# ligne 
p_train_step = pjit(train_step, ...)

# https://github.com/borisdayma/dalle-mini/blob/main/tools/train/train.py, lignes 1051-1055
# train_loader is simply the whole dataset bit by bit
train_loader = dataset.dataloader("train")

# le code de dataloader est ingérable :(
# mais le plus gros du travail semble être fait ici
for idx in batch_idx:
    batch = dataset[idx] # <------------------------------------------------- dataset ?
    batch = {k: jnp.array(v) for k, v in batch.items()}
    yield batch

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4f3692ed-5f27-49a4-899a-82a03e72232c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>